In [8]:
import os, cv2, keyboard, sys
import random
import numpy as np
import pyautogui
import mss
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.relative_locator import locate_with
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, StaleElementReferenceException, ElementNotInteractableException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
url="https://anketa.minzdrav.gov.ru/stacionar/2170c959-4e72-4d32-9a88-4d8ab66bfa0a/4397b2bc-46ca-4206-9f23-ba9716511f0d"

import yandexdriver_update
yandexdriver_update.yandexdriver_update()
tormoz = 5

def start_driver():
    binary_yandex_driver_file = 'yandexdriver.exe'
    
    # Create the profile directory if it doesn't exist:
    profile_directory = os.path.join(os.getcwd(), "NOKUser")
    os.makedirs(profile_directory, exist_ok=True)
    
    options = webdriver.ChromeOptions()
    
    # scale setup:
    # options.add_argument("--window-size=1600,1200")
    options.add_argument("force-device-scale-factor=0.5");
    options.add_argument("high-dpi-support=0.5");
    
    #hide automation from remote site:
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    # using dedicated profile dir:
    options.add_argument(f"--user-data-dir={profile_directory}")
    
    driver = webdriver.Chrome(options=options, service=Service(executable_path=binary_yandex_driver_file))
    
    driver.set_page_load_timeout(60)
    driver.get(url)

    time.sleep(10)
    print('current site:',driver.title)
    return driver

class ClickError(Exception):
    def __init__(self, xpath: str):
        self.xpath = xpath
        self.message = f"Failed to click element with xpath: {xpath}"
        super().__init__(self.message)
        
# def start_driver():
#     binary_yandex_driver_file = 'yandexdriver.exe'
    
#     # Create the profile directory if it doesn't exist:
#     profile_directory = os.path.join(os.getcwd(), "NOKUser")
#     os.makedirs(profile_directory, exist_ok=True)
    
#     options = webdriver.ChromeOptions()
    
#     # Get monitor information
#     with mss.mss() as sct:
#         # Get all monitors
#         monitors = sct.monitors
#         if len(monitors) < 2:
#             print("Warning: Only one monitor detected")
#             monitor = monitors[1]  # fallback to primary monitor
#         else:
#             monitor = monitors[2]  # monitors[1] is primary, [2] is secondary
    
#     # Calculate position for second monitor (adjust these values as needed)
#     window_width = 1600
#     window_height = 1200
#     window_x = monitor["left"]# + 100  # 100px from left edge of second monitor
#     window_y = monitor["top"]# + 100   # 100px from top edge of second monitor
    
#     # Window setup:
#     # options.add_argument(f"--window-size={window_width},{window_height}")
#     # options.add_argument(f"--window-position={window_x},{window_y}")
#     options.add_argument("force-device-scale-factor=0.5")
#     options.add_argument("high-dpi-support=0.5")
    
#     # Other options:
#     options.add_experimental_option("excludeSwitches", ["enable-automation"])
#     options.add_experimental_option('useAutomationExtension', False)
#     options.add_argument("--disable-blink-features=AutomationControlled")
#     options.add_argument(f"--user-data-dir={profile_directory}")
    
#     driver = webdriver.Chrome(options=options, service=Service(executable_path=binary_yandex_driver_file))
    
#     # Additional measures to bring to foreground
#     try:
#         # Method 1: Use pyautogui (may work on Windows)
#         time.sleep(2)  # Give window time to appear
#         pyautogui.getActiveWindow().activate()
        
#         # Method 2: Use Windows API (Windows only)
#         import ctypes
#         hwnd = driver.current_window_handle
#         ctypes.windll.user32.ShowWindow(hwnd, 9)  # SW_RESTORE = 9
#         ctypes.windll.user32.SetForegroundWindow(hwnd)
#     except Exception as e:
#         print(f"Warning: Could not bring window to front: {str(e)}")
    
#     driver.set_page_load_timeout(60)
#     driver.get(url)
    
#     print('current site:', driver.title)
#     return driver

# start_driver()
    
def locate_and_click_matching_images(reference_image_path, confidence=0.8, monitor_num=1, region=None, click_displacement_x=0.5, click_displacement_y=0.5,sleep_after=1):
    """
    Locate and click all instances of a reference image on a specific monitor within a region of interest.
    
    Parameters:
    - reference_image_path: Path to the reference image file
    - confidence: Matching threshold (0-1), higher means more strict matching
    - monitor_num: Monitor number to capture (1 for primary monitor)
    - region: Region of interest as (x, y, width, height) or None for full monitor
    
    Returns:
    - List of coordinates where matches were found and clicked
    """
    # Load reference image
    reference = cv2.imread(reference_image_path)
    if reference is None:
        print(f"Error: Could not load reference image from {reference_image_path}")
        return []
    
    # Convert reference to grayscale
    reference_gray = cv2.cvtColor(reference, cv2.COLOR_BGR2GRAY)
    
    # Get monitor information
    with mss.mss() as sct:
        # Get monitor dimensions
        monitor = sct.monitors[monitor_num]
        
        # Define region of interest
        if region:
            x, y, width, height = region
            monitor = {
                "top": monitor["top"] + y,
                "left": monitor["left"] + x,
                "width": width,
                "height": height
            }
        
        # Capture screenshot
        screenshot = np.array(sct.grab(monitor))
        
    # Convert screenshot to grayscale
    screenshot_gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2RGB)
    screenshot_gray = cv2.cvtColor(screenshot_gray, cv2.COLOR_RGB2GRAY)
    
    # Perform template matching
    result = cv2.matchTemplate(screenshot_gray, reference_gray, cv2.TM_CCOEFF_NORMED)
    
    # Find locations where match quality exceeds our confidence threshold
    locations = np.where(result >= confidence)
    
    # Get reference image dimensions
    ref_height, ref_width = reference_gray.shape
    
    # Store all match locations
    matches = []
    for pt in zip(*locations[::-1]):  # Switch columns and rows
        matches.append(pt)
    
    # Group overlapping matches
    grouped_matches = []
    while matches:
        match = matches.pop(0)
        grouped_matches.append(match)
        
        # Remove any overlapping matches
        non_overlapping = []
        for m in matches:
            # Check if this match overlaps with the current one
            if abs(m[0] - match[0]) > ref_width/2 or abs(m[1] - match[1]) > ref_height/2:
                non_overlapping.append(m)
        matches = non_overlapping
    
    # Click on each match
    clicked_positions = []
    for match in grouped_matches:
        # Calculate center of the match
        center_x = match[0] + ref_width * click_displacement_x
        center_y = match[1] + ref_height * click_displacement_y
        
        # Adjust coordinates to screen position
        screen_x = center_x + monitor["left"]
        screen_y = center_y + monitor["top"]
        
        # Move mouse and click
        pyautogui.moveTo(screen_x, screen_y, duration=0.2)
        pyautogui.click()
        
        clicked_positions.append((screen_x, screen_y))
        time.sleep(0.5)  # Small delay between clicks
    print(f"Found and clicked {len(clicked_positions)} matching images")
    print(f"Clicked positions: {clicked_positions}")
    time.sleep(sleep_after)  # Small delay after click
    if len(clicked_positions)>0:
        return True
    return False

def check_for_error(driver):
    browser_logs = driver.get_log("performance") # логи очищаются после метода get_log()
    for entry in browser_logs:
        try:
            message_obj = json.loads(entry.get("message"))
            message = message_obj.get("message")
            method = message.get("method")
            if method == 'Network.responseReceived':
                # print(message)
                response_url = message.get('params',{}).get('response',{}).get('url','')
                # if 'loadEvnVizitPLForm' in response_url:
                request_id = message.get('params', {}).get('requestId', '')
                print(request_id, end=', ')
                response = driver.execute_cdp_cmd('Network.getResponseBody',{'requestId':request_id})
                response_body = response.get('body','')
                if 'Не верно, попробуйте еще раз!' in str(response_body):
                    print("Не верно, попробуйте еще раз!", response_body)
                    return True
        except Exception as e:
            print('check_for_error failed:')#, e)
    print('No errors!')
    return False

def generate_level_ups(num):
    text=''
    for i in range(num):
        text+="/.."
    return text

def click_xpath(xpath,wait=10,crit=True):
    print("click_xpath", "crit", crit, xpath, end='')
    all_elements = driver.find_elements(By.XPATH, xpath)
    clickable_elements=[]
    for element in all_elements:
        try:
            # Check if element is displayed and enabled
            if element.is_displayed() and element.is_enabled():
                # Additional check - element should be in the viewport and not covered
                is_in_viewport = driver.execute_script("""
                    var elem = arguments[0];
                    var rect = elem.getBoundingClientRect();
                    return (
                        rect.top >= 0 &&
                        rect.left >= 0 &&
                        rect.bottom <= (window.innerHeight || document.documentElement.clientHeight) &&
                        rect.right <= (window.innerWidth || document.documentElement.clientWidth)
                    );
                """, element)
                
                # Check if element is not covered by another element
                is_not_covered = driver.execute_script("""
                    var elem = arguments[0];
                    var rect = elem.getBoundingClientRect();
                    var x = rect.left + rect.width / 2;
                    var y = rect.top + rect.height / 2;
                    var elementAtPoint = document.elementFromPoint(x, y);
                    return elementAtPoint === elem || elem.contains(elementAtPoint);
                """, element)
                
                if is_in_viewport and is_not_covered:
                    clickable_elements.append(element)
        except (ElementNotVisibleException, ElementNotInteractableException):
            # Skip elements that throw visibility or interactability exceptions
            continue
        except Exception as e:
            print(f"Error checking element: {str(e)}")
            continue
    print(f' found {len(clickable_elements)} elements')
    if len(clickable_elements)<1:
        if crit:
            raise ClickError(xpath)
        return False
    for element in clickable_elements:
        try:
            element.click()
            return True
        except (ElementClickInterceptedException, StaleElementReferenceException):
            print("click_xpath intercepted", "crit", crit, xpath)
            time.sleep(1)
            if crit==True:
                counter=0
                
                while counter<3:
                    if click_xpath(xpath,wait=1,crit=False):
                        return True
                    counter+=1
                print("!!!!!!!!!!ClickError Exception!!!!!!!!!!", "crit", crit, xpath)
                raise ClickError(xpath)
            else:
                return False
        except (TimeoutException):
            print("!!!!!!!!!!click_xpath timeout!!!!!!!!!!", "crit", crit, xpath)
            if crit:
                raise ClickError(xpath)
            return False

def type_xpath(xpath,string,wait=10):
    print('typing', string, xpath)
    element = WebDriverWait(driver,wait).until(expected_conditions.element_to_be_clickable((By.XPATH, xpath)))
    time.sleep(0.3)
    element.clear()
    time.sleep(0.3)
    element.send_keys(string)
    time.sleep(1)
    return True
                                 
def click_id(id, wait=10, crit=True):
    xpath = "//*[@id='"+id+"']"
    return click_xpath(xpath, wait=wait, crit=crit)

def click_contains_text(text,level_ups=0, wait=10, crit=True):
    xpath = "//*[contains(text(),'"+ text+"')]"+generate_level_ups(level_ups)
    return click_xpath(xpath, wait=wait, crit=crit)
    
def click_text(text,level_ups=0, wait=10, crit=True):
    xpath = "//*[text()='"+ text+"']"+generate_level_ups(level_ups)
    return click_xpath(xpath, wait=wait, crit=crit)

def click_class(text, wait=10, crit=True, count=0):
    if count==0:
        number=""
    else:
        number="["+str(count)+"]"
    xpath = "(//*[contains(@class,'"+text+"')]"+number+")"
    return click_xpath(xpath, wait=wait, crit=crit)

def randomizator(probability):
    rnd=random.randrange(1,100)
    if rnd <= probability:
        print("yes")
        return 1
    else:
        print("---------------------------Noooooooooooooo!!!")
        return 0
def answer(yes):
    time.sleep(0.2)
    # time.sleep(tormoz)
    if yes:
        click_text('Да')
    else:
        click_text('Нет')
    time.sleep(tormoz)
    nxt()
    
def randomAnswer(probability):
    rnd=random.randrange(1,100)
    if rnd <= probability:
        print("yes")
        answer(1)
        return 1
    else:
        print("---------------------------Noooooooooooooo!!!")
        answer(0)
        return 0

def nxt():
    click_text(' Далее ')
    time.sleep(tormoz)

def send_anketa():
    print("1")#------------question 1-----------------
    if randomizator(1):
        print("---------------------------bingo!!!")
        click_text('плановая')
        nxt()#------------------------------------to question 1.1
        click_text('менее 7 календарных дней')
        nxt()#------------------------------------to question 1.2
        answer(1)#--------------------------------to question 1.3
        click_text('при обращении в медицинскую организацию')
        nxt()#------------------------------------to question 2 
        print("2")#------------question 2-----------------
        if not randomAnswer(95):
            badvariant=random.randrange(1,101)
            if badvariant<10:
                click_text('отсутствие свободных мест ожидания')
            elif badvariant<20:
                click_text('состояние гардероба')
            elif badvariant<30:
                click_text('состояние санитарно-гигиенических помещений')
            elif badvariant<40:
                click_text('отсутствие питьевой воды')
            else:
                click_text('санитарное состояние помещений')
            nxt()#-------------------------------to question 3
        print("3") #------------question 3-----------------
        randomAnswer(95)#---------------------to question 4
    else:
        click_text('экстренная')
        nxt()#------------------------------------to question 4
    print("4")#------------question 4---------------------
    if randomAnswer(3):
        print("4.1")#------------question 4.1-----------------
        print("---------------------------bingo!!!")
        invalid=random.randrange(1,100)
        if invalid < 20:
            click_text('I группа')
        elif invalid < 40:
            click_text('II группа')
        else:
            click_text('III группа')
        nxt()
        #------------question 4.2-----------------
        randomAnswer(95)
        #------------question 4.3-----------------
        randomAnswer(95)
    print("5")#------------question 5-----------------
    if randomAnswer(95):
        #------------question 5.1-----------------
        randomAnswer(95)
    print("6")#------------question 6-----------------
    if randomAnswer(95):
        #------------question 6.1-----------------
        randomAnswer(95)
    print("7")#------------question 7-----------------
    if not randomAnswer(95):
        #------------question 7.1-----------------
            badvariant=random.randrange(1,101)
            time.sleep(tormoz)
            if badvariant<30:
                click_text('питание')
            elif badvariant<40:
                click_text('отсутствие питьевой воды')
            elif badvariant<70:
                click_text('санитарное состояние помещений')
            elif badvariant<80:
                click_text('действия персонала по уходу')
            else:
                click_text('состояние санитарно-гигиенических помещений')
            nxt()
    print("8")#------------question 8-----------------
    randomAnswer(95)
    print("9")#------------question 9-----------------
    randomAnswer(95)
    print("10")#------------question 10-----------------
    randomAnswer(95)
    print("11")#------------question 11-----------------
    randomAnswer(95)
    print("12")#------------question 12-----------------
    if randomAnswer(95):
        #------------question 12.1-----------------
        randomAnswer(95)
    time.sleep(tormoz)
    click_text(' Отправить ')
    print("Clicking send")
    time.sleep(tormoz)
    click_text(' ОК ')
    print("Click OK")
    return True

def maybe_execute(func, probability=0.5, *args, **kwargs):
    """
    Execute a function with a given probability.
    
    Parameters:
    - func: The function to execute
    - probability: Float between 0 and 1 representing execution probability
    - *args, **kwargs: Arguments to pass to the function
    
    Returns:
    - Function result if executed, None otherwise
    """
    if random.random() < probability:
        return func(*args, **kwargs)
    return None

current dir: C:\Users\User\Anaconda
Current version: v25.6.0-stable
Latest version: v25.6.0-stable
Already at the latest version, skipping download


In [9]:
driver = start_driver()
monitor_number = 2
region_of_interest = (50, 50, 600, 600)  # (x, y, width, height) Leave as None to search the entire monitor
if not locate_and_click_matching_images("captcha.png", confidence=0.9, monitor_num=monitor_number, region=region_of_interest,click_displacement_x=0.1,click_displacement_y=0.7):
    sys.exit(1)
locate_and_click_matching_images("medic.png", confidence=0.85, monitor_num=monitor_number, region=region_of_interest)
locate_and_click_matching_images("ready.png", confidence=0.9, monitor_num=monitor_number, region=region_of_interest)
while True:
    if not locate_and_click_matching_images("captcha_error.png", confidence=0.9, monitor_num=monitor_number, region=region_of_interest):
        print('No error!')
        break
    print('Error!')
    locate_and_click_matching_images("medic.png", confidence=0.85, monitor_num=monitor_number, region=region_of_interest)
    locate_and_click_matching_images("ready.png", confidence=0.9, monitor_num=monitor_number, region=region_of_interest)
while True:
    send_anketa()
maybe_execute(send_anketa, 0.5)
maybe_execute(send_anketa, 0.5)
driver.quit()

current site: Анкета НОК
Found and clicked 1 matching images
Clicked positions: [(-1172.2, 584.5)]
Found and clicked 3 matching images
Clicked positions: [(-1136.0, 620.0), (-1072.0, 620.0), (-1008.0, 621.0)]
Found and clicked 1 matching images
Clicked positions: [(-1131.5, 764.0)]
Found and clicked 0 matching images
Clicked positions: []
No error!
1
---------------------------Noooooooooooooo!!!
click_xpath crit True //*[text()='экстренная'] found [<selenium.webdriver.remote.webelement.WebElement (session="963f24f5ab697738eaec7f05e1a48eb5", element="f.B885345B1E57DE0D7FA5FCEC82332113.d.091DF23D2E2BDDBE7586757CF6282166.e.65")>] elements
 found [<selenium.webdriver.remote.webelement.WebElement (session="963f24f5ab697738eaec7f05e1a48eb5", element="f.B885345B1E57DE0D7FA5FCEC82332113.d.091DF23D2E2BDDBE7586757CF6282166.e.77")>] elements
4
---------------------------Noooooooooooooo!!!
 found [<selenium.webdriver.remote.webelement.WebElement (session="963f24f5ab697738eaec7f05e1a48eb5", element

MaxRetryError: HTTPConnectionPool(host='localhost', port=47604): Max retries exceeded with url: /session/963f24f5ab697738eaec7f05e1a48eb5/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000000000AC2CF0>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))